In [1]:
#2.1
i=0
myList = range(3000)
L = sc.parallelize(myList)
#print(L.take(3000))

#2.2

C = L.map(lambda number : number*number*number)
print(C.take(10))
#C.take(300)

[0, 1, 8, 27, 64, 125, 216, 343, 512, 729]

In [2]:
#2.2

C = L.map(lambda number : number*number*number)
print(C.take(10))
print(C.reduce(lambda number1,number2 : number1+number2))
#C.take(300)


[0, 1, 8, 27, 64, 125, 216, 343, 512, 729]
20236502250000

In [3]:
#2.3
result = C.map(lambda number : (number%10,1)).reduceByKey(lambda a,b :a+b)
print(result.take(10))

[(0, 300), (8, 300), (1, 300), (9, 300), (2, 300), (3, 300), (4, 300), (5, 300), (6, 300), (7, 300)]

In [4]:
def digits(i):
  return [e for e in str(i)]
#print(digits(323))
result = C.flatMap(lambda number : digits(number)).map(lambda number : (number,1)).reduceByKey(lambda number1,number2 : number1+number2 ).take(300)
print(result)

[('4', 2762), ('7', 2787), ('1', 3667), ('8', 2639), ('0', 3127), ('9', 2521), ('5', 2653), ('3', 2814), ('6', 2713), ('2', 3294)]

In [5]:
#3.1
myList = range(3000)
C = sc.parallelize(myList)
D = C.cartesian(C)
#D.take(3000)


In [6]:
#3.2
K = 3000
def filterFunction(x,y):
  result = (2*x+1)*(2*x+1) +(2*y+1)*(2*y+1)
  if result < (2*K)*(2*K) :
      return True
  return False
D.take(5)
D1 = D.filter(lambda pair : filterFunction(pair[0],pair[1]))
#print(result)


In [7]:
#3.3
result1 = D.count()
result2 = D1.count()
print(result1)
ratio = D1.count()/D.count()
print(ratio*4)

9000000
3.1415933333333332

In [8]:
#4.2
import re
future_pattern = re.compile("""([^,"]+|"[^"]+")(?=,|$)""")
def parseCSV(line):
  return future_pattern.findall(line)

path_data = "/FileStore/tables/" # CHANGE ME
ratingsFile = sc.textFile(path_data+"/ratings.csv").map(parseCSV)
moviesFile = sc.textFile(path_data+"/movies.csv").map(parseCSV)

print(moviesFile.take(2))

[['movieId', 'title', 'genres'], ['1', 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy']]

In [9]:
#4.3
ratingHeader = ratingsFile.first()
moviesHeader = moviesFile.first() #extract header

ratingLines = ratingsFile.filter(lambda line : line != ratingHeader)
moviesLines = moviesFile.filter(lambda line : line != moviesHeader)

ratingLines=ratingLines.map(lambda l : (l[0],l[1],float(l[2]),l[3]))
ratingLines.take(10)

Out[ 14 ]: [('1', '1', 4.0, '964982703'),
 ('1', '3', 4.0, '964981247'),
 ('1', '6', 4.0, '964982224'),
 ('1', '47', 5.0, '964983815'),
 ('1', '50', 5.0, '964982931'),
 ('1', '70', 3.0, '964982400'),
 ('1', '101', 5.0, '964980868'),
 ('1', '110', 4.0, '964982176'),
 ('1', '151', 5.0, '964984041'),
 ('1', '157', 5.0, '964984100')]

In [10]:
#4.4
#contains the the top 10 rated movies
ratingRdd = ratingLines.map(lambda line : (line[1],(line[2],1))).reduceByKey(lambda a,b : (a[0]+b[0],a[1]+b[1])).map(lambda p : (p[0],p[1][0]/p[1][1])).sortBy(lambda p : -p[1])
ratingRdd.take(10)



Out[ 15 ]: [('158882', 5.0),
 ('60737', 5.0),
 ('27704', 5.0),
 ('175431', 5.0),
 ('25906', 5.0),
 ('136834', 5.0),
 ('104780', 5.0),
 ('134095', 5.0),
 ('1310', 5.0),
 ('72692', 5.0)]

In [11]:
#4.5

mapMovieTitle=moviesLines.map(lambda m : (m[0],m[1]))
movieAverageRating=mapMovieTitle.join(ratingRdd).map(lambda pair : pair[1])
bestMovies = movieAverageRating.sortBy(lambda m : -m[1])
bestMovies.take(10)


Out[ 22 ]: [('Watching the Detectives (2007)', 5.0),
 ('Battle Royale 2: Requiem (Batoru rowaiaru II: Chinkonka) (2003)', 5.0),
 ('Nasu: Summer in Andalusia (2003)', 5.0),
 ('Winnie Pooh (1969)', 5.0),
 ('Black Tar Heroin: The Dark End of the Street (2000)', 5.0),
 ('"Wings, Legs and Tails (1986)"', 5.0),
 ('Human (2015)', 5.0),
 ('Hollywood Chainsaw Hookers (1988)', 5.0),
 ('All Yours (2016)', 5.0),
 ('Loving Vincent (2017)', 5.0)]

In [12]:
#4.6
import math
ratingRdd = ratingLines.map(lambda line : (line[1],(line[2],1))).reduceByKey(lambda a,b : (a[0]+b[0],a[1]+b[1])).map(lambda p : (p[0],p[1][0]/p[1][1]+1)).sortBy(lambda p : -p[1])
print("****raking using the first metric is :****")
print(ratingRdd.take(10))
print("/n/n****raking using the second metric is :****")

ratingRdd = ratingLines.map(lambda line : (line[1],(line[2],1))).reduceByKey(lambda a,b : (a[0]+b[0],a[1]+b[1])).map(lambda p : (p[0],(p[1][0]/p[1][1])*math.log(2))).sortBy(lambda p : -p[1])
 
print(ratingRdd.take(10))  

****raking using the first metric is :****
[('158882', 6.0), ('60737', 6.0), ('27704', 6.0), ('175431', 6.0), ('25906', 6.0), ('136834', 6.0), ('104780', 6.0), ('134095', 6.0), ('1310', 6.0), ('72692', 6.0)]
/n/n****raking using the second metric is :****
[('158882', 3.4657359027997265), ('60737', 3.4657359027997265), ('27704', 3.4657359027997265), ('175431', 3.4657359027997265), ('25906', 3.4657359027997265), ('136834', 3.4657359027997265), ('104780', 3.4657359027997265), ('134095', 3.4657359027997265), ('1310', 3.4657359027997265), ('72692', 3.4657359027997265)]

In [13]:
#5

user1RatedMovies = ratingLines.filter(lambda line : int(line[0]) == 1).map(lambda line : (line[1],line[2]))#.reduceByKey(lambda a,b : a+b)
mapMovieTitle.join(user1RatedMovies).map(lambda pair : pair[1]).take(10)


Out[ 54 ]: [('Braveheart (1995)', 4.0),
 ('Big (1988)', 4.0),
 ('Flight of the Navigator (1986)', 4.0),
 ('Tommy Boy (1995)', 5.0),
 ('Grumpy Old Men (1993)', 5.0),
 ('Gladiator (2000)', 5.0),
 ('Rushmore (1998)', 5.0),
 ('Mission: Impossible (1996)', 3.0),
 ('Canadian Bacon (1995)', 5.0),
 ('"Great Mouse Detective, The (1986)"', 5.0)]